# Drought Classification Workflow

This notebook demonstrates the drought classification workflow for the Snow Drought Index package. It covers loading SSWEI data, classifying drought conditions, analyzing drought characteristics, and visualizing drought patterns over time.

In [4]:
# Import required packages
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from pathlib import Path
import sys
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans


# Import snowdroughtindex package
from snowdroughtindex.core import data_preparation, sswei, drought_classification
from snowdroughtindex.utils import visualization, io

## 1. Load SWE and Precipitation Data

First, we'll load the SWE and precipitation data that are used to classify snow drought explained in Heldmyer et al. CaSR v3.1 data are used in this workflow. onset_to_peak.csv dataset created in 0.3_CaSE_dat_preparation is used here. The dataset consists of SWE and precipitation data extracted from onset (day with 5% of climatological mean SWE) to peak SWE day extracted for the each grid point inside the River Basin.

In [5]:
# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

In [9]:
# Paths for input and output data
CaSR_path = project_root / 'data' / 'output_data' / 'modelled_data_snowdrought' / 'onset_to_peak_data.csv'
peak_SWE_path = project_root / 'data' / 'output_data' / 'modelled_data_snowdrought' / 'peak_SWE_by_elev.csv'
Bow_basin_path = project_root / 'data' / 'input_data' / 'Elevstion' / 'Bow_elevation_combined.shp'
output_data = project_root / 'data' / 'output_data' / 'Heldmyer_method'
output_plots = project_root / 'data' / 'output_plots' / 'Heldmyer_method'


In [8]:
# Load SSWEI data from CSV file
CaSR_data = pd.read_csv(CaSR_path)

# Display the first few rows of the data
CaSR_data.head()

,time,Grid_id,lon,lat,SWE,Precipitation,Elevation_Category,Season_year,daily_SWE_change,cumulative_P
0,1981-02-11,1,-116.13797,51.33459,152.250,0.000000,2000_2500m,1980,0.570312,0.000000
1,1980-11-07,1,-116.13797,51.33459,27.000,2.128011,2000_2500m,1980,0.000000,2.128011
2,1980-12-21,1,-116.13797,51.33459,110.625,3.271602,2000_2500m,1980,0.000000,5.399613
3,1980-10-25,1,-116.13797,51.33459,12.625,0.000000,2000_2500m,1980,27.875000,5.399613
4,1980-11-13,1,-116.13797,51.33459,40.500,0.007695,2000_2500m,1980,13.000000,5.407308


# 2. Find 30 year climatological mean peak SWE for each elevation level

In [10]:
# Load the peak SWE data
peak_SWE_data = pd.read_csv(peak_SWE_path)

# Display the first few rows of the peak SWE data
peak_SWE_data.head()

,Year,peak_1000_1500m,peak_1500_2000m,peak_2000_2500m,peak_500_1000m
0,1981,24.797070,44.364788,154.827127,28.219308
1,1982,26.354167,44.508929,161.058333,40.208705
2,1983,30.337500,38.036621,161.275000,46.500000
3,1984,28.929175,33.191089,130.524913,34.360491
4,1985,17.887549,36.879576,142.178972,55.231249


In [12]:
# Melt the DataFrame to long format for elevation bands
peak_SWE_long = peak_SWE_data.melt(id_vars='Year', 
								   value_vars=[col for col in peak_SWE_data.columns if col.startswith('peak_')],
								   var_name='elevation', 
								   value_name='peak_SWE')

# Calculate mean peak SWE for each elevation band
mean_peak_SWE = peak_SWE_long.groupby('elevation')['peak_SWE'].mean().reset_index()
mean_peak_SWE.columns = ['elevation', 'mean_peak_SWE']

# Display the first few rows of the mean peak SWE data
mean_peak_SWE.head()

,elevation,mean_peak_SWE
0,peak_1000_1500m,31.683571
1,peak_1500_2000m,42.490168
2,peak_2000_2500m,146.697722
3,peak_500_1000m,42.381356


## 3. Identify Snow Drought years

The snow drought are defined as years with peak SWE < climatological peak SWE.

In [14]:
# Find peak SWE for each grid point and each season year
CaSR_data['peak_SWE'] = CaSR_data.groupby(['Grid_id', 'Season_year'])['SWE'].transform('max')

# keep only season year, grid point, elevation category and peak SWE columns
CaSR_peak_SWE = CaSR_data[['Season_year', 'Grid_id', 'Elevation_Category', 'peak_SWE']].drop_duplicates()

display(CaSR_peak_SWE.head())

,Season_year,Grid_id,Elevation_Category,peak_SWE
0,1980,1,2000_2500m,240.625
203,1981,1,2000_2500m,277.000
424,1982,1,2000_2500m,273.875
614,1983,1,2000_2500m,200.375
794,1984,1,2000_2500m,227.750


In [17]:
# get mean peak SWE for each elevation category and each season year
mean_peak_SWE_by_elev = CaSR_peak_SWE.groupby(['Season_year', 'Elevation_Category'])['peak_SWE'].mean().reset_index()

# pivot so that each elevation category is a column, indexed by season year
mean_peak_SWE_by_elev = mean_peak_SWE_by_elev.pivot(index='Season_year', columns='Elevation_Category', values='peak_SWE')

display(mean_peak_SWE_by_elev)

Elevation_Category,1000_1500m,1500_2000m,2000_2500m,500_1000m
Season_year,,,,
1980,40.117253,56.006445,157.767231,41.898997
1981,29.995833,45.382143,167.513889,43.205357
1982,17.079167,41.866078,163.965798,54.445312
1983,30.662508,39.921101,139.377604,45.846590
1984,23.369173,46.719413,145.626281,71.951294
1985,33.612565,64.917292,220.556988,147.183036
1986,24.904169,48.265046,138.041667,81.272409
1987,25.312655,46.318848,142.251107,61.542703
1988,39.641667,62.800140,174.979123,73.125000


## 3. Visualize Drought Classifications

Let's visualize the drought classifications using the plotting functions from the drought classification module.

In [ ]:
# Plot default drought classification
fig1 = drought_classification.plot_drought_classification(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI',
    classification_column='Drought_Classification'
)
plt.title('Default Drought Classification')
plt.show()

# Plot custom drought classification
fig2 = drought_classification.plot_drought_classification(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI',
    classification_column='Custom_Classification'
)
plt.title('Custom Drought Classification')
plt.show()

## 4. Calculate Drought Characteristics

Now we'll calculate drought characteristics such as duration, severity, and intensity for each drought event.

In [ ]:
# Calculate drought characteristics using default threshold (-0.5)
drought_chars = drought_classification.calculate_drought_characteristics(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI'
)

# Display drought characteristics
if not drought_chars.empty:
    print("Drought Characteristics:")
    display(drought_chars)
else:
    print("No drought events found in the data.")

## 5. Visualize Drought Characteristics

Let's visualize the drought characteristics to better understand the drought events.

In [ ]:
# Plot drought characteristics
if not drought_chars.empty:
    fig3 = drought_classification.plot_drought_characteristics(drought_chars)
    plt.show()
else:
    print("No drought events to plot.")

## 6. Analyze Drought Trends

Let's analyze drought trends over time using a moving window approach.

In [ ]:
# Define window size for trend analysis
window_size = 10  # 10-year moving window

# Analyze drought trends
trend_data = drought_classification.analyze_drought_trends(
    sswei_data,
    year_column='season_year',
    swei_column='SWEI',
    window_size=window_size
)

# Display trend data
if not trend_data.empty:
    print(f"Drought Trends (using {window_size}-year moving window):")
    display(trend_data.head())
else:
    print("Not enough data for trend analysis.")

## 7. Visualize Drought Trends

Let's visualize the drought trends to identify any changes in drought frequency and severity over time.

In [ ]:
# Plot drought trends
if not trend_data.empty:
    fig4 = drought_classification.plot_drought_trends(trend_data)
    plt.show()
else:
    print("Not enough data for trend visualization.")

## 8. Compare Drought Classifications by Decade

Let's analyze how drought classifications have changed by decade.

In [ ]:
# Add decade column
sswei_data['decade'] = (sswei_data['season_year'] // 10) * 10

# Count classifications by decade
decade_counts = pd.crosstab(sswei_data['decade'], sswei_data['Drought_Classification'])

# Display counts
print("Drought Classifications by Decade:")
display(decade_counts)

# Plot heatmap of classifications by decade
plt.figure(figsize=(12, 8))
sns.heatmap(decade_counts, cmap='YlOrRd', annot=True, fmt='d', cbar_kws={'label': 'Count'})
plt.title('Drought Classifications by Decade')
plt.ylabel('Decade')
plt.xlabel('Drought Classification')
plt.tight_layout()
plt.show()

## 9. Analyze Drought Severity Distribution

Let's analyze the distribution of drought severity values.

In [ ]:
# Calculate drought severity for all years
sswei_data['drought_severity'] = sswei_data['SWEI'].apply(drought_classification.get_drought_severity)

# Plot histogram of drought severity
plt.figure(figsize=(10, 6))
sns.histplot(sswei_data[sswei_data['drought_severity'] > 0]['drought_severity'], bins=10, kde=True)
plt.title('Distribution of Drought Severity')
plt.xlabel('Drought Severity')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate summary statistics for drought severity
severity_stats = sswei_data[sswei_data['drought_severity'] > 0]['drought_severity'].describe()
print("Drought Severity Statistics:")
print(severity_stats)

## 10. Save Drought Analysis Results

Finally, let's save the drought analysis results for future reference.

In [ ]:
# Save drought characteristics
if not drought_chars.empty:
    drought_chars.to_csv('../data/processed/drought_characteristics.csv', index=False)
    print("Drought characteristics saved to '../data/processed/drought_characteristics.csv'")

# Save drought trends
if not trend_data.empty:
    trend_data.to_csv('../data/processed/drought_trends.csv', index=False)
    print("Drought trends saved to '../data/processed/drought_trends.csv'")

# Save SSWEI data with custom classification and severity
sswei_data.to_csv('../data/processed/sswei_with_drought_analysis.csv', index=False)
print("SSWEI data with drought analysis saved to '../data/processed/sswei_with_drought_analysis.csv'")

## 11. Summary

In this notebook, we've demonstrated the drought classification workflow for the Snow Drought Index package. We've loaded SSWEI data, classified drought conditions using both default and custom thresholds, analyzed drought characteristics, visualized drought patterns, and examined drought trends over time.

The workflow uses the following key functions from the `drought_classification` module:
- `classify_drought()` for classifying drought conditions based on SSWEI values
- `calculate_drought_characteristics()` for analyzing drought events
- `analyze_drought_trends()` for examining changes in drought patterns over time
- Various plotting functions for visualizing drought classifications and characteristics

These functions provide a comprehensive toolkit for analyzing drought conditions based on SSWEI data, enabling researchers to better understand drought patterns, severity, and trends over time.